### **Transform Team Dimension**

### Configuration

In [1]:
%run /utils/general_functions

In [2]:
create_mounts()

### Load Data

In [3]:
job_id = mssparkutils.env.getJobId()

team_df = spark.read.format('delta').load(f'synfs:/{job_id}/mnt/silver/dim_team')

### Transformation

In [4]:
from pyspark.sql.functions import when, col

team_final_df = team_df.withColumn('abbreviation', when(col('abbreviation').isNull(), 'N/a').otherwise(col('abbreviation'))) \
    .withColumn('city', when(col('city').isNull(), 'N/a').otherwise(col('city'))) \
    .withColumn('conference', when(col('conference').isNull(), 'N/a').otherwise(col('conference'))) \
    .withColumn('division', when(col('division').isNull(), 'N/a').otherwise(col('division'))) \
    .withColumn('full_name', when(col('full_name').isNull(), 'N/a').otherwise(col('full_name'))) \
    .withColumn('name', when(col('name').isNull(), 'N/a').otherwise(col('name'))) 

### Merge Data

In [5]:
container = 'gold'
database = 'prize_picks_gold'
table = 'dim_team'
file_format = 'delta'
merge_condition = 'tgt.team_key = src.team_key'

merge_data(team_final_df, container, database, table, file_format, merge_condition=merge_condition )

In [ ]:
%%sql
DROP TABLE IF EXISTS prize_picks_silver.dim_team;